In [1]:
from itertools import chain, combinations
import numpy as np # Fundamental package for scientific computing with Python
import pandas as pd
import calendar
import time
from sklearn.metrics import accuracy_score, precision_score, f1_score
from lib.Utility import computeEquity, getData
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split


commissioni = 0.0005


In [2]:
# Train the model
def trainModel(X_train, y_train):
    # Set regularization rate
    reg = 0.01
    # train a logistic regression model on the training set
    mod = LogisticRegression(C=1/reg, solver="liblinear", random_state=42).fit(X_train, y_train)

    return mod

In [3]:
data = getData()

features = ['feat_Open-PrevClose', 'feat_vvixPrevOpen', 'feat_vvixPrevHigh', 'feat_vvixPrevHigh-Low', 'feat_vvixPrevLow'           # , 'feat_Open'
            , 'feat_vixOpen', 'feat_vixPrevClose'
            , 'feat_vvixOpen', 'feat_vvixPrevClose'
            # ,   'feat_Acc', 'feat_PrevCluster'
            , 'feat_bigBodyRange']

combinazioni = chain.from_iterable(combinations(
    features, r) for r in range(1, len(features) + 1))

combos = []
for idx, combinazione in enumerate(combinazioni):
    current_GMT = time.gmtime()
    time_stamp = calendar.timegm(current_GMT)

    print(idx, time.strftime("%H:%M:%S", time.localtime()), ':', combinazione)
    combos.append(combinazione)

dataframe = pd.DataFrame(columns=['DataSetName', 'TrainAccuracy', 'TrainPrecision', 'TrainF2', 'TrainRecall',
                         'TrainROC', 'TestAccuracy', 'TestPrecision', 'TestF2', 'TestRecall', 'TestROC', 'BestParams'])

split_test_date = '2019-02-21'
start_date = '1990-09-04'

# Split Dataset
print("Performing splitting operations....")
# train_data = data[(data['Date'] < split_test_date) & (data['Date'] > start_date)].reset_index(drop=True)
# val_data = data[(data['Date'] < split_test_date) & (data['Date'] > start_date)].reset_index(drop=True)
# test_data = data[data['Date'] >= split_test_date].reset_index(drop=True)

# Select 'feat' columns
print("Extracting features columns and creating target variable...")

print(combos)

for idx, combo in enumerate(combos):
    current_GMT = time.gmtime()
    time_stamp = calendar.timegm(current_GMT)
    print(idx, time.strftime("%H:%M:%S", time.localtime()),
          'Trying with following combo', combo)
    feat_cols = [col for col in combo if 'feat' in col]
    # Import Training and Validation Data
    # x_train = train_data[feat_cols]
    # y_train = np.where(train_data['Close'] > train_data['Open'], 1, -1)

    # x_test = test_data[feat_cols]
    # y_test = np.where(test_data['Close'] > test_data['Open'], 1, -1)

    x_train, x_test, y_train, y_test = train_test_split(
        data[feat_cols], data['Cluster'], test_size=0.3, random_state=1, shuffle=False)

    x_train = x_train.sort_index()
    x_test = x_test.sort_index()
    y_train = y_train.sort_index()
    y_test = y_test.sort_index()

    model = trainModel(x_train, y_train)

    # Predictions
    y_train_pred = model.predict(x_train.sort_index())
    y_test_pred = model.predict(x_test.sort_index())
    text_pred_prob = model.predict_proba(x_test)
    # print('y_test_pred', y_test_pred)
    # clf.fit(x_train,y_train)

    # y_train_pred = clf.predict(x_train)
    # y_test_pred = clf.predict(x_test)

    # VN Calcolo la equity

    test_data = data[data.index.isin(x_test.index)]
    # print('test_data',test_data)

    test_data['cluster'] = y_test_pred
    test_equity, test_cagr, test_max_dd = computeEquity(
        test_data, commissioni, feat_cols, text_pred_prob)

    print("Train metrics...")
    # Metrics
    # train_accuracy = accuracy_score(y_train, y_train_pred)
    # train_precision = precision_score(y_train, y_train_pred)

    print("Test metrics...")
    test_accuracy = accuracy_score(y_test, y_test_pred)
    # test_precision = precision_score(y_test, y_test_pred)
    # test_f1 = f1_score(y_test, y_test_pred)

    # Save Results
    new_row = pd.Series({
        'DataSetName': combo,
        # 'TrainAccuracy': train_accuracy,
        # 'TrainPrecision': train_precision,
        'TestAccuracy': test_accuracy,
        # 'TestPrecision': test_precision,
        # 'TestF1': test_f1,
        'BestParams': model.get_params,
        'TestEquity': test_equity,
        'TestCagr': test_cagr,
        'TestMaxDD': test_max_dd
    })
    print(new_row)
    dataframe = pd.concat([dataframe, new_row.to_frame().T], ignore_index=True)

dataframe.to_excel("Dataframe/"+str(time_stamp) + ".xlsx")

# print(result)

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
C:\Users\vitto\AppData\Local\Temp\ipykernel_18820\3500483520.py:75: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_data['cluster'] = y_test_pred


0 19:30:52 : ('feat_Open-PrevClose',)
1 19:30:52 : ('feat_vvixPrevOpen',)
2 19:30:52 : ('feat_vvixPrevHigh',)
3 19:30:52 : ('feat_vvixPrevHigh-Low',)
4 19:30:52 : ('feat_vvixPrevLow',)
5 19:30:52 : ('feat_vixOpen',)
6 19:30:52 : ('feat_vixPrevClose',)
7 19:30:52 : ('feat_vvixOpen',)
8 19:30:52 : ('feat_vvixPrevClose',)
9 19:30:52 : ('feat_bigBodyRange',)
10 19:30:52 : ('feat_Open-PrevClose', 'feat_vvixPrevOpen')
11 19:30:52 : ('feat_Open-PrevClose', 'feat_vvixPrevHigh')
12 19:30:52 : ('feat_Open-PrevClose', 'feat_vvixPrevHigh-Low')
13 19:30:52 : ('feat_Open-PrevClose', 'feat_vvixPrevLow')
14 19:30:52 : ('feat_Open-PrevClose', 'feat_vixOpen')
15 19:30:52 : ('feat_Open-PrevClose', 'feat_vixPrevClose')
16 19:30:52 : ('feat_Open-PrevClose', 'feat_vvixOpen')
17 19:30:52 : ('feat_Open-PrevClose', 'feat_vvixPrevClose')
18 19:30:52 : ('feat_Open-PrevClose', 'feat_bigBodyRange')
19 19:30:52 : ('feat_vvixPrevOpen', 'feat_vvixPrevHigh')
20 19:30:52 : ('feat_vvixPrevOpen', 'feat_vvixPrevHigh-Low')

C:\Users\vitto\AppData\Local\Temp\ipykernel_18820\3500483520.py:75: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_data['cluster'] = y_test_pred


Train metrics...
Test metrics...
DataSetName                                  (feat_vvixPrevOpen,)
TestAccuracy                                             0.542489
BestParams      <bound method BaseEstimator.get_params of Logi...
TestEquity                                            -1332.72551
TestCagr                                              -242.681428
TestMaxDD                                            -1669.175405
dtype: object
2 19:30:54 Trying with following combo ('feat_vvixPrevHigh',)
Equity: -1332.7255097656266


C:\Users\vitto\AppData\Local\Temp\ipykernel_18820\3500483520.py:75: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_data['cluster'] = y_test_pred


...

Train metrics...
Test metrics...
DataSetName     (feat_Open-PrevClose, feat_vvixPrevHigh, feat_...
TestAccuracy                                             0.554628
BestParams      <bound method BaseEstimator.get_params of Logi...
TestEquity                                            3787.813064
TestCagr                                               689.738343
TestMaxDD                                             -631.852442
dtype: object
1021 19:45:45 Trying with following combo ('feat_vvixPrevOpen', 'feat_vvixPrevHigh', 'feat_vvixPrevHigh-Low', 'feat_vvixPrevLow', 'feat_vixOpen', 'feat_vixPrevClose', 'feat_vvixOpen', 'feat_vvixPrevClose', 'feat_bigBodyRange')
Equity: 3286.734939453135


C:\Users\vitto\AppData\Local\Temp\ipykernel_18820\3500483520.py:75: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_data['cluster'] = y_test_pred


Train metrics...
Test metrics...
DataSetName     (feat_vvixPrevOpen, feat_vvixPrevHigh, feat_vv...
TestAccuracy                                             0.550835
BestParams      <bound method BaseEstimator.get_params of Logi...
TestEquity                                            3286.734939
TestCagr                                               598.494981
TestMaxDD                                             -887.609546
dtype: object
1022 19:45:46 Trying with following combo ('feat_Open-PrevClose', 'feat_vvixPrevOpen', 'feat_vvixPrevHigh', 'feat_vvixPrevHigh-Low', 'feat_vvixPrevLow', 'feat_vixOpen', 'feat_vixPrevClose', 'feat_vvixOpen', 'feat_vvixPrevClose', 'feat_bigBodyRange')
Equity: 3790.0738066406334


C:\Users\vitto\AppData\Local\Temp\ipykernel_18820\3500483520.py:75: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_data['cluster'] = y_test_pred


Train metrics...
Test metrics...
DataSetName     (feat_Open-PrevClose, feat_vvixPrevOpen, feat_...
TestAccuracy                                             0.552352
BestParams      <bound method BaseEstimator.get_params of Logi...
TestEquity                                            3790.073807
TestCagr                                                690.15001
TestMaxDD                                             -878.756104
dtype: object
